# Sentiment Analysis

We will now see how to classify documents based on their sentiment.

## IMDb movie review dataset

The dataset consists of 50,000 polar movie reviews labeled as positive or negative. Now we will load the dataset as a **DataFrame**

In [3]:
import pyprind
import pandas as pd
import os

basepath = "/home/alanmarazzi/Scaricati/aclImdb"

# Create progress bar for data loading
pbar = pyprind.ProgBar(50000)
labels = {'pos': 1, 'neg': 0}
df = pd.DataFrame()

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()

df.columns = ['review', 'sentiment']

0%                          100%
[##############################] | ETA: 00:01:21 | ETA: 00:01:19 | ETA: 00:01:18 | ETA: 00:01:17 | ETA: 00:01:18 | ETA: 00:01:17 | ETA: 00:01:16 | ETA: 00:01:14 | ETA: 00:01:12 | ETA: 00:01:10 | ETA: 00:01:08 | ETA: 00:01:06 | ETA: 00:01:05 | ETA: 00:01:05 | ETA: 00:01:02 | ETA: 00:01:01 | ETA: 00:00:58 | ETA: 00:00:54 | ETA: 00:00:49 | ETA: 00:00:45 | ETA: 00:00:40 | ETA: 00:00:36 | ETA: 00:00:31 | ETA: 00:00:27 | ETA: 00:00:22 | ETA: 00:00:18 | ETA: 00:00:13 | ETA: 00:00:09 | ETA: 00:00:04 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:02:20


After loading the dataset we have to clean it, in fact class labels are sorted and we don't want this since we have to split the set in training and test.

In [4]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./movie_data.csv', index=False)

After shuffling we saved the dataset as csv, so it's going to be easier to work with it.

In [5]:
df = pd.read_csv('./movie_data.csv')
df.head(3)

,review,sentiment
0,<br /><br />There is something about seeing a ...,1
1,I fail to understand why anyone would allow a ...,0
2,Disney has yet to meet a movie it couldn't mak...,0


## Bag-of-words model

